简单例子:

重点关注那些save/load，还有一些附加操作的依赖,global_step。


In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
from PIL import Image
import numpy as np

In [2]:
INPUT_SIZE = 784
OUTPUT_SIZE = 10
L1_SIZE = 500

BATCH_SIZE = 200
LEARNING_RATE_BASE = 0.1
LEARNING_RATE_DECAY = 0.99
REGULAR_SCALE = 0.0001
STEPS = 50000
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH = './model/'
MODEL_NAME = 'mnist_model'

DATA_DIR = 'MNIST_data'

In [3]:
def get_weight(shape, regular_scale):
    w = tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    if regular_scale != None:
        tf.add_to_collection('regular_loss',tf.contrib.layers.l2_regularizer(regular_scale)(w))
    return w
def get_bias(shape):
    b = tf.Variable(tf.zeros(shape))
    return b
def net(x,regular_scale):
    w1 = get_weight([INPUT_SIZE,L1_SIZE], regular_scale)
    b1 = get_bias([L1_SIZE])
    l1 = tf.nn.relu(tf.matmul(x,w1)+b1)
    
    w2 = get_weight([L1_SIZE, OUTPUT_SIZE], regular_scale)
    b2 = get_bias([OUTPUT_SIZE])
    y = tf.matmul(l1,w2)+b2
    return y  

In [29]:
def train(data):
    x = tf.placeholder(tf.float32, [None,INPUT_SIZE])
    y_ = tf.placeholder(tf.float32, [None,OUTPUT_SIZE])
    y = net(x,REGULAR_SCALE)
    global_step = tf.Variable(0, trainable = False)
    
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = y, labels = tf.argmax(y_,1))#label需要转换成非one-hot
    cem = tf.reduce_mean(ce)
    loss = cem + tf.add_n(tf.get_collection('regular_loss'))
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,data.train.num_examples / BATCH_SIZE,
                                              LEARNING_RATE_DECAY, staircase = True)
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    
    ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    ema_op = ema.apply(tf.trainable_variables())
    with tf.control_dependencies([train_op, ema_op]):
        train_op = tf.no_op(name='train')#train_op重名，也可以改名
    
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter('model_mnist_ema_ckpt_graph')
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        #断点续训：
        ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
        
        for i in range(STEPS):
            xs, ys = data.train.next_batch(BATCH_SIZE)
            _, loss_val, step = sess.run([train_op,loss,global_step],feed_dict={x:xs,y_:ys})
            if i % 1000 == 0:
                print('after %d steps, total loss is %g'%(step,loss_val))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)
                

In [30]:
def main():
    mnist = input_data.read_data_sets(DATA_DIR,one_hot=True)
    train(mnist)
if __name__ == '__main__':
    main()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from ./model/mnist_model-49001
after 49002 steps, total loss is 0.126964
after 50002 steps, total loss is 0.124086
after 51002 steps, total loss is 0.122567
after 52002 steps, total loss is 0.127575
after 53002 steps, total loss is 0.120838
after 54002 steps, total loss is 0.125351


KeyboardInterrupt: 

In [54]:
#测试
#假设这是独立的文件和进程，变量肯定要读出来
#这几个读取方式交叉互动有点乱。
#saver是拿变量的，利用ckpt从路径找到了具体带后缀的文件名
#global_step直接就是文件名的隐含信息。
#这里主要是拿ema，ckpt的之后再说

#因为带ema的模型，预测就只需要拿ema就行了（但是ema没有bias吧，怎么办？），而拿ckpt的所有参数主要是训练期间的续训。。。
#根据这句代码，ema肯定包含了bias，甚至global_step都要专门处理，才能防止被ema
#ema_op = ema.apply(tf.trainable_variables())

import tensorflow as tf
import time
TEST_INTERVAL_SECS = 5
def test(data):
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32, [None, INPUT_SIZE])
        y_ = tf.placeholder(tf.float32, [None, OUTPUT_SIZE])
        y = net(x,None)#Net结构是直接读取的定义，ema的变量都是从这找的
        
        ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        #这是一套映射：如果变量有ema，就读ema，否则，用普通。
        #用ema resotre来初始化saver，影响saver.restore的读取内容吗？也就是说，能读取bias吗？
        #根据形状判断（因为变量名不可读），应该是有bias的
        ema_restore = ema.variables_to_restore()
        #打印的含义？为何有Exponential，是已经读到了？还是按规则自己假设的变量名？应该是按规则预设的变量名
        print('ema_restore length:',len(ema_restore))
        print('ema_restore:',ema_restore)
        saver = tf.train.Saver(ema_restore)
        print('saver:',saver)#<class 'tensorflow.python.training.saver.Saver'>
        print('saver print end')
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
#         while True:#把无秒一次改为总共一次
        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
            
            #从路径里读出来一个'checkpoint'文件，这个文件就包含了最新的model文件名和最近N个model文件名。
            #字典形式：model_checkpoint_path，all_model_checkpoint_paths，下边已经打印。
            print('ckpt type:', type(ckpt))
            print('------------------ckpt obj print start-------------------\n',ckpt)
            print('------------------ckpt obj print end--------------------------')
            print('ckpt all_model_checkpoint_paths type:',type(ckpt.all_model_checkpoint_paths))#算是protobuf的一个容器
            print('ckpt all_model_checkpoint_paths:',ckpt.all_model_checkpoint_paths)
            for i in range(len(ckpt.all_model_checkpoint_paths)):
                print('%dth path is %s'%(i,ckpt.all_model_checkpoint_paths[i]))
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)#他只是从proto拿了路径，还是自己的restore
                global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                accuracy_score = sess.run(accuracy,feed_dict={x:data.test.images,y_:data.test.labels})
                print('after % steps, accuracy is %f'%(global_step,accuracy_score))
                
                print('ckpt.model_checkpoint_path:',ckpt.model_checkpoint_path)#./model/mnist_model-49001
                print('ckpt path print end')
                
                print('all trainable variables:', tf.trainable_variables())
                
            else:
                print('no ckpt')
                return
#             time.sleep(TEST_INTERVAL_SECS)
def main():
    mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)
    test(mnist)
if __name__ == '__main__':
    main()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
ema_restore length: 4
ema_restore: {'Variable_1/ExponentialMovingAverage': <tf.Variable 'Variable_1:0' shape=(500,) dtype=float32_ref>, 'Variable/ExponentialMovingAverage': <tf.Variable 'Variable:0' shape=(784, 500) dtype=float32_ref>, 'Variable_2/ExponentialMovingAverage': <tf.Variable 'Variable_2:0' shape=(500, 10) dtype=float32_ref>, 'Variable_3/ExponentialMovingAverage': <tf.Variable 'Variable_3:0' shape=(10,) dtype=float32_ref>}
saver: <tensorflow.python.training.saver.Saver object at 0x7fbfe82a0470>
saver print end
ckpt type: <class 'tensorflow.python.training.checkpoint_state_pb2.CheckpointState'>
------------------ckpt obj print start-------------------
 model_checkpoint_path: "./model/mnist_model-54002"
all_model_checkpoint_paths: "./model/mnist_model-50002"
all_model_checkpoint_paths: 

# 下面输入手写数字图片，进行预处理并进行预测。

In [32]:
#from PIL import Image

In [63]:
#同上面的测试阶段，主要是恢复EMA
#他起名太烂，这何止是restore？restore之后也没返回给外界，这是restore之后直接原地预测
#而且地址都是写死的，这严格讲，只能算预测
#拿mnist的test去测，准确率98%，但是自己手写还是差强人意。

MNIST_TEST = False


def predict(testPicArr):
    with tf.Graph().as_default() as tg:
        x = tf.placeholder(tf.float32, [None, INPUT_SIZE])
        y_ = tf.placeholder(tf.float32, [None, OUTPUT_SIZE])
        y = net(x,None)
        pred = tf.argmax(y,1)
        
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)
        
        if MNIST_TEST:
            mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)
            correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                
                if MNIST_TEST:
                    accuracy_score = sess.run(accuracy,feed_dict={x:mnist.test.images,y_:mnist.test.labels})
                    print('accuracy is %f'%(accuracy_score))
                
                preValue = sess.run(pred, feed_dict={x:testPicArr})
                return preValue
            else:
                print('no ckpt file')
                return -1
            
            
#关于threshold，为了能使用网络图片，改成手动输入
#灰度图改成极端情况，0是黑的255是白的，这个可以调。比如我的彩色图都是一百几到二百，可能就得弄一百大几十
#如果是网上随便down的图，可能每个图都要设置不一样的参数，或者用平均值来代替？
#最好是统一形式的数据
def preprocess_picture(picDir,threshold=180):
    img = Image.open(picDir)
#     print(img)
#     print(img.mode, img.size, img.format)
#     img.show()
    resized_img = img.resize((28,28), Image.ANTIALIAS)#好像是防锯齿

    resized_img.save(picDir.split('.')[0]  +'resize' + '.png')#保存一下，但是下一步变ndarray，不能用Image的接口保存了。
    
    img_arr = np.array(resized_img.convert('L'))#单色，灰度
    print(img_arr.shape)
#     print(img_arr)

    for i in range(28):
        for j in range(28):
            if (img_arr[i][j] < threshold):
                img_arr[i][j] = 0
            else:
                img_arr[i][j] = 255
    nm_arr = img_arr.reshape([1,784])#变成mnist形式
    nm_arr = nm_arr.astype(np.float32)
    img_ready = np.multiply(nm_arr, 1.0 / 255.0)#255变成1
    
    return img_ready
    
            
#png的是RGBA，反正最终要黑白的，np.array之后就是单通道了。

png_2 = preprocess_picture('mnist_pic_test/2.png',255)#windows画图的黑色手写，区分点基本就是255了。
png_3 = preprocess_picture('mnist_pic_test/3.png',255)#windows画图的黑色手写，区分点基本就是255了。

print(predict(png_2))#预测成了8.。。。。
print(predict(png_3))#预测成了6.。。。。

#换两个粗的，写的可能太细了。
print(predict(preprocess_picture('mnist_pic_test/2_thick.png',255)))#预测成了5.。。。。
print(predict(preprocess_picture('mnist_pic_test/3_thick.png',255)))#预测成了8.。。。。

#这个刷子好像不纯黑，多涂了几次
#稍微接近一些了
print(predict(preprocess_picture('mnist_pic_test/2_thick_black.png',255)))#预测成了2.。。。。
print(predict(preprocess_picture('mnist_pic_test/3_thick_black.png',255)))#预测成了5.。。。。

#用纯黑笔写,写紧凑，截图时放大，就变粗了，
#效果也不太好。
print(predict(preprocess_picture('mnist_pic_test/2_enlarge.png',255)))#预测成了5.。。。。
print(predict(preprocess_picture('mnist_pic_test/3_enlarge.png',255)))#预测成了5.。。。。

#来俩简单的
print(predict(preprocess_picture('mnist_pic_test/1_enlarge.png',255)))#预测成了4.。。。。
print(predict(preprocess_picture('mnist_pic_test/4_enlarge.png',255)))#预测成了2.。。。。
print(predict(preprocess_picture('mnist_pic_test/0_enlarge.png',255)))#预测成了5.。。。。
print(predict(preprocess_picture('mnist_pic_test/7_enlarge.png',255)))#预测成了5.。。。。
#万物皆可55555555

#放弃，放弃，数据和模型果然得花功夫好好调才能匹配的上。
#因为mnist本身就很简单，28*28,网络结构也简单，一个隐层。
#手写字体是西方人的写法，自己用鼠标在画板上画的可能也不标准，所以误差大。
#也不至于偏差这么大吧？可能颜色反了，网上搜mnist有白字有黑字，白字效果也一般。

print('white:')
# print(predict(preprocess_picture('mnist_pic_test/1_white.png',255)))#预测成了4.。。。。
# print(predict(preprocess_picture('mnist_pic_test/2_white.png',255)))#预测成了4.。。。。
print(predict(preprocess_picture('mnist_pic_test/3_white.png',255)))#预测成了5.。。。。
print(predict(preprocess_picture('mnist_pic_test/4_white.png',255)))#预测成了7.。。。。
# print(predict(preprocess_picture('mnist_pic_test/5_white.png',255)))#预测成了4.。。。。
print(predict(preprocess_picture('mnist_pic_test/6_white.png',255)))#预测成了3.。。。。
print(predict(preprocess_picture('mnist_pic_test/7_white.png',255)))#预测成了3.。。。。
print(predict(preprocess_picture('mnist_pic_test/8_white.png',255)))#预测成了8.。。。。
print(predict(preprocess_picture('mnist_pic_test/9_white.png',255)))#预测成了7.。。。。


#另一个可能的问题，我的手写字体都是比较高的，28*28应该是正方形，所以，多少有点缩放，原则上区别也没这么大，就当写矮一点，也差不多

#使用mnist自带的图片测试
print(predict(preprocess_picture('MNIST_data/raw/mnist_train_16.jpg',255)))#预测成了2,原图也是2
print(predict(preprocess_picture('MNIST_data/raw/mnist_train_23.jpg',255)))#预测成了1,原图是1
print(predict(preprocess_picture('MNIST_data/raw/mnist_train_17.jpg',255)))#预测成了7,原图是9,也不绝对准确




(28, 28)
(28, 28)
INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
[8]
INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
[6]
(28, 28)
INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
[5]
(28, 28)
INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
[8]
(28, 28)
INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
[2]
(28, 28)
INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
[5]
(28, 28)
INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
[5]
(28, 28)
INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
[5]
(28, 28)
INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
[4]
(28, 28)
INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
[2]
(28, 28)
INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
[5]
(28, 28)
INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
[5]
white:
(28, 28)
INFO:tensorf

In [49]:
#help(tf.Graph().as_default) #返回的是上下文manager，
#     This method should be used if you want to create multiple graphs
#     in the same process. For convenience, a global default graph is
#     provided, and all ops will be added to this graph if you do not
#     create a new graph explicitly. Use this method with the `with` keyword
#     to specify that ops created within the scope of a block should be
#     added to this graph.
    
#     The default graph is a property of the current thread. If you
#     create a new thread, and wish to use the default graph in that
#     thread, you must explicitly add a `with g.as_default():` in that
#     thread's function.

#下面两个用法相等
#用法1：声明一个graph，as_default
g = tf.Graph()

with g.as_default():
  c = tf.constant(5.0)
  print(c.graph is g)
d = tf.constant(6.0)
print(d.graph)
print(g)
print(d.graph is g)#显式声明的g不是全局默认
# print(g.Session())

#用法2：直接as
with tf.Graph().as_default() as g2:
    c = tf.constant(5.0)
    print(c.graph is g2)
    print(c.graph is g)#无关
    print(g2)

print(tf.Graph())#每个都是一个生成的
print(tf.Graph())

# g和g2和默认graph都不一样

True
False


AttributeError: 'Graph' object has no attribute 'Session'

In [5]:
help(ckpt.model_checkpoint_path)

NameError: name 'ckpt' is not defined

In [ ]:
variables_to_restore(moving_avg_variables=None) method of tensorflow.python.training.moving_averages.ExponentialMovingAverage instance
    Returns a map of names to `Variables` to restore.
    
    If a variable has a moving average, use the moving average variable name as
    the restore name; otherwise, use the variable name.
    
    For example,
    
    ```python
      variables_to_restore = ema.variables_to_restore()
      saver = tf.train.Saver(variables_to_restore)
    ```
    
    Below is an example of such mapping:
    
    ```
      conv/batchnorm/gamma/ExponentialMovingAverage: conv/batchnorm/gamma,
      conv_4/conv2d_params/ExponentialMovingAverage: conv_4/conv2d_params,
      global_step: global_step
    ```
    Args:
      moving_avg_variables: a list of variables that require to use of the
        moving variable name to be restored. If None, it will default to
        variables.moving_average_variables() + variables.trainable_variables()
    
    Returns:
      A map from restore_names to variables. The restore_name can be the
      moving_average version of the variable name if it exist, or the original
      variable name.

In [10]:
help(tf.train.Saver)

Help on class Saver in module tensorflow.python.training.saver:

class Saver(builtins.object)
 |  Saves and restores variables.
 |  
 |  See @{$variables$Variables}
 |  for an overview of variables, saving and restoring.
 |  
 |  The `Saver` class adds ops to save and restore variables to and from
 |  *checkpoints*.  It also provides convenience methods to run these ops.
 |  
 |  Checkpoints are binary files in a proprietary format which map variable names
 |  to tensor values.  The best way to examine the contents of a checkpoint is to
 |  load it using a `Saver`.
 |  
 |  Savers can automatically number checkpoint filenames with a provided counter.
 |  This lets you keep multiple checkpoints at different steps while training a
 |  model.  For example you can number the checkpoint filenames with the training
 |  step number.  To avoid filling up disks, savers manage checkpoint files
 |  automatically. For example, they can keep only the N most recent files, or
 |  one checkpoint for e

In [13]:
help(tf.train.saver.restore)

AttributeError: module 'tensorflow.python.training.training' has no attribute 'saver'

In [12]:
help(tf.train.get_checkpoint_state)

Help on function get_checkpoint_state in module tensorflow.python.training.saver:

get_checkpoint_state(checkpoint_dir, latest_filename=None)
    Returns CheckpointState proto from the "checkpoint" file.
    
    If the "checkpoint" file contains a valid CheckpointState
    proto, returns it.
    
    Args:
      checkpoint_dir: The directory of checkpoints.
      latest_filename: Optional name of the checkpoint file.  Default to
        'checkpoint'.
    
    Returns:
      A CheckpointState if the state was available, None
      otherwise.
    
    Raises:
      ValueError: if the checkpoint read doesn't have model_checkpoint_path set.

